from example https://github.com/reichenbch/RAG-examples/blob/main/LangChain%20LLamaIndex%20RAG.ipynb

In [1]:
%load_ext autoreload
%autoreload 2
import sys,torch, os
sys.path.insert(0, '/home/sulcan/Documents/ipac-logbook/code/')
from desy_wrapper import logbook_to_documents
from gradio_wrapper import *
import configparser

config = configparser.ConfigParser()
config.read('../config/XFELelog.ini')

device = ('cuda' if torch.cuda.is_available() else 'cpu')
data_folder = config['sources']['data_folder']
data_file = config['sources']['data_file']
faiss_index = config['sources']['faiss_index']

In [2]:
embedding = prepare_embedding()

if os.path.isfile(f'{faiss_index}/index.faiss'):
    vectorstore = FAISS.load_local(faiss_index, embedding, allow_dangerous_deserialization=True)
else:
    data = logbook_to_documents(data_file)
    vectorstore = prepare_vectorstore(data.values(), embedding)
    vectorstore.save_local(faiss_index)

retriever = prepare_retriever(vectorstore)
tokenizer, model = prepare_llm(device = device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
query = 'Compile a list of all BPMs at I1?'

In [5]:
query, answer = generate(query, retriever, tokenizer, model, device)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [6]:
print(query,answer)

Compile a list of all BPMs at I1?  Based on the context provided, the following is a list of BPMs at I1:

* BPMF.48.I1
* BPMA.92.I1
* BPMA.117.I1
* BPMA.134.L1
* BPMA.158.L1
* BPMA.179.B1
* BPMA.103.I1
* BPMA.105.I1
* BPMA.107.I1
* BPMA.110.I1
* BPMA.112.I1
* BPMA.115.I1
* BPMA.117.I1
* BPMA.119.I1

Please note that some of these BPMs were set on different dates, as indicated in the context.


In [14]:

# config = ConfigParser.ConfigParser()

'/home/sulcan/Documents/logbook/data_local/'
